In [1]:
!pip install -qq transformers

     |████████████████████████████████| 665kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 8.9MB/s 
     |████████████████████████████████| 1.1MB 34.1MB/s 
     |████████████████████████████████| 890kB 50.8MB/s 


In [2]:
!pip install igramscraper

     |████████████████████████████████| 81kB 3.3MB/s 
  Created wheel for igramscraper: filename=igramscraper-0.3.5-cp36-none-any.whl size=36738 sha256=354af38bc37ab1e5c0d7a713630e68ed4c68796e79da0b1897f76ea377136162
  Stored in directory: /root/.cache/pip/wheels/25/1d/ec/610e51c7273336de5778876c6e84c9fed146da6d3e273191d9
  Created wheel for python-slugify: filename=python_slugify-3.0.2-py2.py3-none-any.whl size=4726 sha256=330f56536bf19ca3b7898aac7865e802242fbf4675e9473764475bb6320b321c
  Stored in directory: /root/.cache/pip/wheels/16/7f/c3/6b0582283ad589d68a306da924a78c74546e010d8106b9b3a9
Successfully built igramscraper python-slugify
  Found existing installation: text-unidecode 1.3
    Uninstalling text-unidecode-1.3:
      Successfully uninstalled text-unidecode-1.3
  Found existing installation: python-slugify 4.0.0
    Uninstalling python-slugify-4.0.0:
      Successfully uninstalled python-slugify-4.0.0


In [0]:
import torch
import pandas as pd
import numpy as np
import sys
from transformers import BertModel, BertTokenizer
from os import path
from google.colab import drive
from torch import nn
from torch.nn import functional as F
import os.path
from os import listdir
from os.path import isfile, join
from tqdm import tqdm


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
drive_root="/content/gdrive/My Drive/"   ###EDIT_HERE -- mostly won't have to tho
sys.path.insert(1, drive_root+'data/scrapper') ###EDIT_HERE -- scrapper files directory
import test_data_generator
from test_data_generator import generate_test_data

In [6]:
MAX_LEN = 300
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [0]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [0]:
folder_path = drive_root + "data/models/" ###EDIT_HERE  -- model directory
file_name = "model_all_12k.h5" ###EDIT_HERE -- model filename
state_dict = torch.load(folder_path + file_name, map_location=torch.device('cpu'))

In [0]:
data_path = drive_root+'data/'
class_names = [os.path.splitext(f)[0] for f in listdir(data_path) if isfile(join(data_path, f))]  #INPUT FILES

In [10]:
model = SentimentClassifier(len(class_names))

In [11]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [0]:
def get_prediction_array(review_text):

  encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
  )
  
  input_ids = encoded_review['input_ids'].to(device)
  attention_mask = encoded_review['attention_mask'].to(device)
  
  output = model(input_ids, attention_mask)
  return output

In [0]:
def get_interests(test_df, threshold = 50):
  predictions=[0] * len(class_names)
  for index in tqdm(range(min(threshold, len(test_df)))):
    caption = test_df["captions"][index]
    prediction=F.softmax(get_prediction_array(caption), dim = 1)
    prediction=prediction[:].detach().cpu().numpy()
    predictions=predictions+prediction
  data_len=len(test_df["captions"])
  if data_len>0 :
    predictions=(1/data_len)*predictions

  return predictions

In [18]:
user_account="skyemcalpine"
test_df=generate_test_data(user_account,100)
print(test_df)

100% (100 of 100) |######################| Elapsed Time: 0:00:00 ETA:  00:00:00


media count1824
account name : skyemcalpine

Dataframe generated
                                            photo_url  ...                                           hashtags
0   https://scontent-hkg4-1.cdninstagram.com/v/t51...  ...               [bestwaytospendtheday, summerpicnic]
1   https://scontent-hkg4-1.cdninstagram.com/v/t51...  ...                                  [sundayafternoon]
2   https://scontent-hkg4-1.cdninstagram.com/v/t51...  ...  [myfavouritekindoffood, frommydiningtable, laz...
3   https://scontent-hkg4-1.cdninstagram.com/v/t51...  ...                      [atableinvenice, weekendmood]
4   https://scontent-hkg4-1.cdninstagram.com/v/t51...  ...                    [thatfridayfeeling, summermood]
..                                                ...  ...                                                ...
95  https://scontent-hkg4-1.cdninstagram.com/v/t51...  ...                                       [autumnmood]
96  https://scontent-hkg4-1.cdninstagram.com/v/t51...  

In [19]:
final_pred = get_interests(test_df, 50)
sorted_args = (-final_pred).argsort()

100%|██████████| 50/50 [00:53<00:00,  1.07s/it]


In [21]:
n = 3  #TOP n INTERESTS
top_n = min(n, len(class_names))  #top n interests
interests = np.array(class_names)[sorted_args].flatten()[:top_n]
probabilities = final_pred[:,sorted_args].flatten()[:top_n]


print(f"Top {n} interests of {user_account}: {interests}")
print(f"\nWith probabilities: {probabilities}")

Top 3 interests of skyemcalpine: ['Food and drink' 'Family and Relationships' 'Sports and Outdoors']

With probabilities: [4.99995300e-01 9.69854517e-07 7.85927431e-07]
